In [2]:
import os
import pandas as pd
import snscrape.modules.twitter as sntwitter
import numpy as np
import datetime
from nltk.tokenize import word_tokenize   
import text_proccessing
from text_proccessing import clean_text
from text_proccessing import remove_stopwords
from text_proccessing import lemmatize_text
from text_proccessing import concatenate_text
from text_proccessing import makeglove
from text_proccessing import make_embedding_matrix

In [3]:
code_dir = os.getcwd()
parent_dir = os.path.dirname(code_dir)
data_dir = os.path.join(parent_dir,"Data")
tweet_dir = os.path.join(parent_dir,"TweetMap")


In [4]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Mon Mar  1 10:40:16 2021

@author: Arbo
"""

def twittsearch(text_query,since_date,until_date,tweetcount):
    
    tweets_list = []

    query = '{0} since:{1} until:{2} filter:has_engagement'.format(text_query, since_date, until_date)
    print(query)

    # Using TwitterSearchScraper to scrape data and append tweets to list

    for i,tweet in enumerate(sntwitter.TwitterSearchScraper(query).get_items()):

        if i>tweetcount:

            break
        tweets_list.append([tweet.date, tweet.id, tweet.content, tweet.url,
                            tweet.lang,tweet.retweetedTweet,tweet.quotedTweet])

    # Creating a dataframe from the tweets list above

    tweets_df = pd.DataFrame(tweets_list, 
                             columns=['Datetime', 'TweetId', 'Text','TweetLink',
                                     "Language", "RTFrom", "QTFrom"])
    
    print("found {} tweets ranging from {} to {}".format(len(tweets_df),
                                                            tweets_df.Datetime.min(),tweets_df.Datetime.max()))
    
    print("dropping duplicates")   
    tweets_df = tweets_df.drop_duplicates(subset=['TweetId'])
    print("total of tweets now: {}".format(len(tweets_df)))
    print("english only")

    tweets_df = tweets_df[tweets_df["Language"]=='en']
    print("total of tweets now: {}".format(len(tweets_df)))
    tweets_df = tweets_df[tweets_df["RTFrom"].isna()]
    print("total of tweets now: {}".format(len(tweets_df)))
    tweets_df = tweets_df[tweets_df["QTFrom"].isna()]
    print("total of tweets now: {} ranging from {} to {}".format(len(tweets_df),
                                                                tweets_df.Datetime.min(),tweets_df.Datetime.max()))
    
    tweets_df = tweets_df[['Datetime', 'TweetId', 'Text','TweetLink']]
    return tweets_df




In [7]:
scrapestart =  datetime.datetime.now() 
print("Scraping commenced at {}".format(scrapestart))
# =============================================================================
text_query = '("forest fire") OR wildfire OR bushfire OR \
(extreme heat) OR (record heat) OR heatwave OR ("heat wave") OR typhoon OR cyclone OR hurricane OR \
tornado OR ("storm surge") OR  blizzard OR snow OR ("ice storm") OR sleet OR thunderstorm OR \
hail OR flood OR flooding OR freeze OR frost OR (extreme cold) OR landslide OR tsunami OR ("tidal wave") OR \
earthquake OR eruption OR volcano OR lava OR lahar OR avalanche OR mudslide OR sinkhole'


# qlist = [firemetq_geologicalotherq]
# qname= ['fire & meteorlogical & geological & other']
since_date = '2021-06-26'
until_date = '2021-06-27'
tweetcount = 200000
twts = twittsearch(text_query,since_date,until_date,tweetcount)
# twittsearch(text_query,since_date,until_date,tweetcount).to_csv(os.path.join(tweet_dir,'tweets_df_{0}_{1}.csv'.
#                               format(since_date.replace('-',''),until_date.replace('-',''))),
#                  index =False)
# =============================================================================
scrapend =  datetime.datetime.now()
print("Scraping ended at {}".format(scrapend))
print("Scraping time {}".format(scrapend-scrapestart))

Scraping commenced at 2021-07-17 15:23:29.888834
("forest fire") OR wildfire OR bushfire OR (extreme heat) OR (record heat) OR heatwave OR ("heat wave") OR typhoon OR cyclone OR hurricane OR tornado OR ("storm surge") OR  blizzard OR snow OR ("ice storm") OR sleet OR thunderstorm OR hail OR flood OR flooding OR freeze OR frost OR (extreme cold) OR landslide OR tsunami OR ("tidal wave") OR earthquake OR eruption OR volcano OR lava OR lahar OR avalanche OR mudslide OR sinkhole since:2021-06-26 until:2021-06-27 filter:has_engagement
found 73977 tweets ranging from 2021-06-26 00:00:00+00:00 to 2021-06-26 23:59:57+00:00
dropping duplicates
total of tweets now: 73977
english only
total of tweets now: 44024
total of tweets now: 44024
total of tweets now: 41043 ranging from 2021-06-26 00:00:00+00:00 to 2021-06-26 23:59:57+00:00
Scraping ended at 2021-07-17 15:37:04.473853
Scraping time 0:13:34.585019


In [8]:
print("processing ")
procstart =  datetime.datetime.now() 
twts['ptext'] = twts['Text'].apply(lambda x: clean_text(x))
twts['ptext'] = twts['ptext'].apply(lambda x: word_tokenize(x))
twts['ptext'] = twts['ptext'].apply(lambda x : remove_stopwords(x))
twts['ptext'] = twts['ptext'].apply(lambda x : lemmatize_text(x))
twts['ptext'] = twts['ptext'].apply(lambda x : concatenate_text(x))
twts.to_csv(os.path.join(tweet_dir,"processed_{}_{}.csv".format(since_date.replace('-',''),until_date.replace('-',''))), index = False)
procend=  datetime.datetime.now()


print("processed {}".format(procend - procstart))



processing 
processed 0:01:54.239296
